# Zestaw zadań: Równania różniczkowe - spectral bias
## Zadanie 1
### Autor: Artur Gęsiarz

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

### Definicja analitycznego rozwiązania

In [ ]:
def exact_solution(x, w):
    return (1/w) * torch.sin(w * x)

### Definicja sieci neuronowej

In [ ]:
class FCN(nn.Module):
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

### Funkcja obliczajca koszt warunku poczatkowego

In [ ]:
def calculate_cond_start_cost(model, x_boundary):
    u0 = model(x_boundary)
    loss_ic = u0 ** 2
    return  loss_ic

### Funkcja obliczajca koszt rezydualny

In [ ]:
def calculate_residual_cost(model, x_physics, w):
    u = model(x_physics)
    du_dx = torch.autograd.grad(u, x_physics, torch.ones_like(u), create_graph=True)[0]
    residual = du_dx - torch.cos(w * x_physics)
    loss_r = torch.mean(residual**2)
    return loss_r

### Funkcja obliczajca koszt totalny

In [ ]:
def calculate_total_cost(model, x_boundary, x_physics, w):
    return calculate_cond_start_cost(model, x_boundary) + calculate_residual_cost(model, x_physics, w)

### Funkcja treningowa PINN

In [ ]:
def train_PINN(model, x_boundary, x_physics, cost_fun, w, epochs=50000, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    losses = []

    for i in range(epochs):
        optimizer.zero_grad()

        loss = cost_fun(model, x_boundary, x_physics, w)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()

        if i % 5000 == 0:
            print(f'Epoch {i}, Loss: {loss.item()}')

    return losses

### Funkcja do rysowania wyników

In [ ]:
def plot_results(x_test, u_exact, u_pred, losses, title):

    plt.plot(x_test, u_exact, label='Exact solution')
    plt.plot(x_test, u_pred, '--', label='PINN solution')
    plt.xlabel('x')
    plt.ylabel('u(x)')
    plt.title(title + ': Solution')
    plt.legend()
    plt.show()

    plt.plot(x_test, abs((u_exact - u_pred)))
    plt.xlabel('x')
    plt.ylabel('error')
    plt.title(title + ': Error function')
    plt.show()

    plt.plot(losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(title + ': Training Loss')
    plt.show()

### Stałe w naszym modelu

In [ ]:
N_INPUT = 1
N_OUTPUT = 1
LR = 0.001
EPOCHS = 50000

### a) Przypadek $ \omega = 1$

### Parametry naszego modelu

In [ ]:
N_LAYERS = 2
N_HIDDEN = 16
TRAINING_POINTS = 200
TESTING_POINTS = 1000
OMEGA = 1

### Definicja modelu

In [ ]:
model = FCN(N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS)

### Definicja punktów treningowych i testowych

In [ ]:
x_boundary = torch.tensor([[0.0]], requires_grad=True)
x_physics = torch.linspace(-2 * np.pi, 2 * np.pi, TRAINING_POINTS).view(-1, 1).requires_grad_(True)
x_test = torch.linspace(-2 * np.pi, 2 * np.pi, TESTING_POINTS).view(-1, 1)
u_exact = exact_solution(x_test, OMEGA)

### Trening modelu

In [ ]:
print(f'Training for w = {OMEGA}, Layers = {N_LAYERS}, Neurons = {N_HIDDEN}\n')
#losses = train_PINN(model, x_boundary, x_physics, calculate_total_cost, OMEGA, EPOCHS, LR)

### Zapisywanie treningu do osobnego pliku

In [ ]:
torch.save(model, "models/model_a_2_16.pth")

### Wczytywanie treningu z pliku

In [ ]:
model_load = torch.load("models/model_a_2_16.pth")

### Przewidywanie wartości

In [ ]:
u_pred = model_load(x_test).detach().numpy()
u_exact = u_exact.numpy()

### Rysowanie wynikow

In [ ]:
#plot_results(x_test, u_exact, u_pred, losses, f'w = {OMEGA}, Layers = {N_LAYERS}, Neurons = {N_HIDDEN}')

### b) Przypadek $ \omega = 15$

### Parametry naszego modelu

In [ ]:
TRAINING_POINTS = 3000
TESTING_POINTS = 5000
OMEGA = 15
ARCH = [(2,16), (4,64), (5,128)]

### Def. modelu, def. pkt. treningowych, def. pkt. testowych, trenig, przewidywanie wartosci, rysowanie...

In [ ]:
# for N_LAYERS, N_HIDDEN in ARCH:
#
#     # Definicja modelu
#     model = FCN(N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS)
#
#     # Definicja punktów treningowych i testowych
#     x_boundary = torch.tensor([[0.0]], requires_grad=True)
#     x_physics = torch.linspace(-2 * np.pi, 2 * np.pi, TRAINING_POINTS).view(-1, 1).requires_grad_(True)
#     x_test = torch.linspace(-2 * np.pi, 2 * np.pi, TESTING_POINTS).view(-1, 1)
#     u_exact = exact_solution(x_test, OMEGA)
#
#     # Trening
#     print(f'Training for w = {OMEGA}, Layers = {N_LAYERS}, Neurons = {N_HIDDEN}\n')
#     losses = train_PINN(model, x_boundary, x_physics, calculate_total_cost,OMEGA, EPOCHS, LR)
#
#     # Zapis
#     torch.save(model, f"models/model_b_{N_LAYERS}_{N_HIDDEN}.pth")
#
#     # Odczyt
#     model_load = torch.load(f"models/model_b_{N_LAYERS}_{N_HIDDEN}.pth")
#
#     # Przewidywanie wartości
#     u_pred = model(x_test).detach().numpy()
#     u_exact = u_exact.numpy()
#
#     # Rysowanie wykresow
#     plot_results(x_test, u_exact, u_pred, losses, f'w = {OMEGA}, Layers = {N_LAYERS}, Neurons = {N_HIDDEN}')

### c) Dla wybranej sieci porównam wynik z rozwiązaniem w ktorym przyjeto inne rozwiazanie

### Funkcja obliczajca koszt totalny

In [ ]:
def calculate_total_cost_anastaz(model, _, x_physics, w):
    u = torch.tanh(w * x_physics) * model(x_physics)
    u_x = torch.autograd.grad(u, x_physics, torch.ones_like(u), create_graph=True)[0]
    loss_r = torch.mean(torch.pow((u_x - torch.cos(w * x_physics)),2))
    return loss_r

### Parametry naszego modelu

In [ ]:
N_LAYERS = 5
N_HIDDEN = 128
TRAINING_POINTS = 3000
TESTING_POINTS = 5000
OMEGA = 15

### Definicja modelu

In [ ]:
# model = FCN(N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS)

### Definicja punktów treningowych i testowych

In [ ]:
x_physics = torch.linspace(-2 * np.pi, 2 * np.pi, TRAINING_POINTS).view(-1, 1).requires_grad_(True)
x_test = torch.linspace(-2 * np.pi, 2 * np.pi, TESTING_POINTS).view(-1, 1)
u_exact = exact_solution(x_test, OMEGA)

### Trening

In [ ]:
print(f'Training for w = {OMEGA}, Layers = {N_LAYERS}, Neurons = {N_HIDDEN}\n')
losses = train_PINN(model, _, x_physics, calculate_total_cost_anastaz, OMEGA, EPOCHS, LR)

### Zapis

In [ ]:
torch.save(model, f"models/model_c_{N_LAYERS}_{N_HIDDEN}.pth")

### Odczyt

In [ ]:
model_c_load = torch.load(f"models/model_c_{N_LAYERS}_{N_HIDDEN}.pth")

### Przewidywanie wartości

In [ ]:
u_pred = model_c_load(x_test).detach().numpy()
u_exact = u_exact.numpy()

### Rysowanie wykresow

In [ ]:
# plot_results(x_test, u_exact, u_pred, losses, f'ansatz - w = {OMEGA}, Layers = {N_LAYERS}, Neurons = {N_HIDDEN}')